In [3]:
!pip install allennlp
!pip install contractions
!pip install torch
!pip install torchinfo
!pip install gensim
!pip install transformers

!mkdir model_storage

mkdir: cannot create directory ‘model_storage’: File exists


In [4]:
#EJECUTAR DOS VECES
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize, casual_tokenize
from nltk.corpus import stopwords
import json
import nltk
import math
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
import numpy as np
from allennlp.modules.elmo import Elmo, batch_to_ids
import contractions
import torch
import os
from argparse import Namespace
from torch.utils.data import Dataset, DataLoader
import tqdm
from tqdm import tqdm_notebook
from torchinfo import summary
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

from gensim.models.keyedvectors import KeyedVectors

#Cargamos un modelo pre-entrenado limitándolo a las 400K palabras más comunes. TODO: recuerda reemplazar el path por la localización del modelo en tu Drive.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP-PROYECTO/Word2Vec/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=400000)

Mounted at /content/drive


In [6]:
dt = pd.read_csv('canciones.csv')

train_data = dt[dt['split'] == 'train'].reset_index(drop=True)
test_data = dt[dt['split'] == 'test'].reset_index(drop=True)

stop_words = set(stopwords.words('english'))
tokenizer = TreebankWordTokenizer()

def split_into_sentences(lyrics):
    sections = lyrics.split('\n')
    sentences = []
    for section in sections:
        if section.strip():
            sentences.extend(sent_tokenize(section.strip()))
    return sentences

def tokenize_lyrics(lyrics):
    tokens = tokenizer.tokenize(lyrics)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens





train_data['sentences'] = train_data['lyrics'].apply(split_into_sentences)
train_data['tokens_palabras'] = train_data['lyrics'].apply(tokenize_lyrics)

test_data['sentences'] = test_data['lyrics'].apply(split_into_sentences)
test_data['tokens_palabras'] = test_data['lyrics'].apply(tokenize_lyrics)



In [7]:
x_train = train_data['lyrics']
y_train = train_data['tag']

x_test = test_data['lyrics']
y_test = test_data['tag']

In [8]:


vectorizer = TfidfVectorizer(min_df=1,stop_words='english')

train_tfidf = vectorizer.fit_transform(x_train)
test_tfidf = vectorizer.transform(x_test)



In [9]:


count_vect = CountVectorizer(tokenizer=casual_tokenize)
X_train_count = count_vect.fit_transform(x_train)
X_test_count = count_vect.transform(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# **TEXT CLASSIFICATION CON SHALLOW**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MaxAbsScaler
import numpy as np

def classify_text(X_train, y_train, X_test, y_test, feature_name, vectorizer):
    # Escalar datos
    scaler = MaxAbsScaler()  # Escala datos dispersos como TF-IDF
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Modelos
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train_scaled, y_train)

    lr = LogisticRegression(random_state=42, solver='saga', max_iter=1000, class_weight='balanced')
    lr.fit(X_train_scaled, y_train)

    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train_scaled, y_train)

    linear_svm = LinearSVC(random_state=42, max_iter=2000)
    linear_svm.fit(X_train_scaled, y_train)

    nonlinear_svm = SVC(random_state=42, kernel='rbf')
    nonlinear_svm.fit(X_train_scaled, y_train)

    # Almacenar modelos y sus predicciones
    models = {
        "RandomForest": rf,
        "Logistic Regression": lr,
        "Decision Tree": dt,
        "Linear SVM": linear_svm,
        "Non-linear SVM": nonlinear_svm
    }

    results = []

    # Métricas por modelo
    for model_name, model in models.items():
        predictions = model.predict(X_test_scaled)

        # Calcular métricas
        acc = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions, average='weighted')
        recall = recall_score(y_test, predictions, average='weighted')
        f1 = f1_score(y_test, predictions, average='weighted')

        # Imprimir resultados detallados
        print(f"Resultados de {model_name} con el texto representado como {feature_name}:")
        print(classification_report(y_test, predictions))
        print("-" * 60)

        # Guardar resultados
        results.append({
            'Modelo': model_name,
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        })

    # Mostrar resultados como tabla
    results_df = pd.DataFrame(results)
    print("\nTabla de Resultados:")
    print(results_df)

    # Palabras importantes para Random Forest
    if hasattr(rf, 'feature_importances_'):
        importances = rf.feature_importances_
        feature_names = vectorizer.get_feature_names_out()
        sorted_indices = np.argsort(importances)[::-1]
        print("\nTop palabras para Random Forest:")
        for i in range(10):
            print(f"{feature_names[sorted_indices[i]]}: {importances[sorted_indices[i]]}")

    # Palabras importantes para Regresión Logística
    if hasattr(lr, 'coef_'):
        coef = lr.coef_[0]
        sorted_indices = np.argsort(coef)[::-1]
        print("\nTop palabras para Regresión Logística:")
        for i in range(10):
            print(f"{feature_names[sorted_indices[i]]}: {coef[sorted_indices[i]]}")


In [ ]:
classify_text(train_tfidf, y_train, test_tfidf, y_test, "TfidfVectorizer", vectorizer)

Resultados de RandomForest con el texto representado como TfidfVectorizer:
              precision    recall  f1-score   support

     country       0.59      0.68      0.64      1129
         pop       0.45      0.41      0.43      1138
         rap       0.86      0.89      0.87      1158
        rock       0.53      0.49      0.51      1075

    accuracy                           0.62      4500
   macro avg       0.61      0.62      0.61      4500
weighted avg       0.61      0.62      0.62      4500

------------------------------------------------------------
Resultados de Logistic Regression con el texto representado como TfidfVectorizer:
              precision    recall  f1-score   support

     country       0.64      0.68      0.66      1129
         pop       0.42      0.41      0.41      1138
         rap       0.87      0.84      0.85      1158
        rock       0.49      0.49      0.49      1075

    accuracy                           0.61      4500
   macro avg       0.

# **CLASES PREVIAS**




In [10]:
#CNN y LSTM
class Vocabulary(object):
    """Clase para procesar texto y extrar el vocabulario existente para su posterior mapeo."""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary

        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index

        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [11]:
#CNN y LSTM
class SequenceVocabulary(Vocabulary):
    """Extensión de la clase vocabulario para lidiar con la secuencialidad del texto."""
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx) #Llamamos al constructor padre.

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        #Usamos el vocabulario inicial y añadimos los nuevos tokens de uso especial.
        self.mask_index = self.add_token(self._mask_token) #Padding
        self.unk_index = self.add_token(self._unk_token) #Tokens desconocidos
        self.begin_seq_index = self.add_token(self._begin_seq_token) #Inicio de una frase.
        self.end_seq_index = self.add_token(self._end_seq_token) #Fin de una frase.

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token
          or the UNK index if token isn't present.

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
              for the UNK functionality
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [12]:
import numpy as np
from collections import Counter
import string
#CNN y LSTM
class SongVectorizer(object):
    """Transforma los tokens en IDs numéricos y añade tokens auxiliares (BEGIN, END, MASK)."""
    def __init__(self, lyrics_vocab, tag_vocab):
        self.lyrics_vocab = lyrics_vocab  # Vocabulario para las letras de las canciones
        self.tag_vocab = tag_vocab  # Vocabulario que representa los géneros (tags) de las canciones

    def vectorize(self, lyrics, vector_length=-1):
        """
        Vectoriza las letras en una representación numérica.
        Args:
            lyrics (str): una cadena con palabras separadas por espacio
            vector_length (int): longitud fija para el vector (relleno con MASK)
        Returns:
            numpy.array: las letras vectorizadas
        """
        indices = [self.lyrics_vocab.begin_seq_index]
        indices.extend(self.lyrics_vocab.lookup_token(token)
                       for token in lyrics.split(" "))
        indices.append(self.lyrics_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        # Vector con ceros y longitud fija
        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.lyrics_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, songs_df, cutoff=5):
        """
        Crea un vectorizador desde el dataframe usando las letras (lyrics).
        Args:
            songs_df (pandas.DataFrame): dataset de canciones
            cutoff (int): frecuencia mínima para incluir un token en el vocabulario
        Returns:
            SongVectorizer: instancia del vectorizador
        """
        songs_df['lyrics'] = songs_df['lyrics'].fillna("").astype(str)

        # Crear vocabulario de géneros (tags)
        tag_vocab = Vocabulary()
        for tag in sorted(set(songs_df.tag)):
            tag_vocab.add_token(tag)

        # Contar palabras en las letras
        word_counts = Counter()
        for lyrics in songs_df.lyrics:
            for token in lyrics.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1

        # Crear vocabulario para las letras
        lyrics_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                lyrics_vocab.add_token(word)

        return cls(lyrics_vocab, tag_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """
        Carga un vectorizador desde un objeto serializable.
        """
        lyrics_vocab = SequenceVocabulary.from_serializable(contents['lyrics_vocab'])
        tag_vocab = Vocabulary.from_serializable(contents['tag_vocab'])
        return cls(lyrics_vocab=lyrics_vocab, tag_vocab=tag_vocab)

    def to_serializable(self):
        """
        Convierte el vectorizador en un formato serializable.
        """
        return {
            'lyrics_vocab': self.lyrics_vocab.to_serializable(),
            'tag_vocab': self.tag_vocab.to_serializable()
        }


In [13]:
#CNN y LSTM
class SongDataset(Dataset):
    def __init__(self, songs_df, vectorizer):
        """
        Args:
            songs_df (pandas.DataFrame): el dataset
            vectorizer (SongVectorizer): vectorizador instanciado desde el dataset
        """
        self.songs_df = songs_df
        self._vectorizer = vectorizer

        # Longitud máxima para las letras con tokens especiales (ejemplo: <BOS>, <EOS>)
        measure_len = lambda lyrics: len(lyrics.split(" "))
        self._max_seq_length = max(map(measure_len, songs_df.lyrics)) + 2

        # Dividir en conjuntos de entrenamiento, validación y prueba
        self.train_df = self.songs_df[self.songs_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.songs_df[self.songs_df.split == 'validation']
        self.validation_size = len(self.val_df)

        self.test_df = self.songs_df[self.songs_df.split == 'test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Pesos para las clases
        class_counts = songs_df.tag.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.tag_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, songs_csv):
        """Carga el dataset y crea un nuevo vectorizador desde cero"""
        songs_df = pd.read_csv(songs_csv)
        train_songs_df = songs_df[songs_df.split == 'train']
        return cls(songs_df, SongVectorizer.from_dataframe(train_songs_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, songs_csv, vectorizer_filepath):
        """Carga el dataset y el vectorizador previamente guardado"""
        songs_df = pd.read_csv(songs_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(songs_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """Carga solo el vectorizador desde un archivo JSON"""
        with open(vectorizer_filepath) as fp:
            return SongVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """Guarda el vectorizador en formato JSON"""
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """Devuelve el vectorizador"""
        return self._vectorizer

    def set_split(self, split="train"):
        """Establece el conjunto actual de datos"""
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """Devuelve un elemento del dataset"""
        row = self._target_df.iloc[index]

        lyrics_vector = self._vectorizer.vectorize(row.lyrics, self._max_seq_length)
        tag_index = self._vectorizer.tag_vocab.lookup_token(row.tag)

        return {'x_data': lyrics_vector,
                'y_target': tag_index}

    def get_num_batches(self, batch_size):
        """Devuelve el número de lotes según el tamaño del batch"""
        return len(self) // batch_size


def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    Generador que envuelve el DataLoader de PyTorch.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {name: tensor.to(device) for name, tensor in data_dict.items()}
        yield out_data_dict


In [14]:
#CNN y LSTM
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        else:
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t


            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

# **Text classification con CNN**

In [22]:
class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification.
    Adapted from: https://www.kaggle.com/code/williamlwcliu/cnn-text-classification-pytorch"""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=100,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=4,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            pretrained_embeddings = torch.from_numpy(pretrained_embedding).float()
            self.emb = nn.Embedding(embedding_dim=embed_dim,
                                    num_embeddings=vocab_size,
                                    padding_idx=0,
                                    _weight=pretrained_embeddings)
        else:
            self.embed_dim = embed_dim
            self.emb = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.emb(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)

        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [29]:
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download


def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def download_embedding_file(repo_id, filename, cache_dir):
    file_path = os.path.join(cache_dir, filename)
    if not os.path.exists(file_path):
        print("Descargando los word embeddings")
        file_path = hf_hub_download(repo_id=repo_id, filename=filename, cache_dir=cache_dir)
    else:
        print("El fichero de embeddings ya está descargado.")
    return file_path


def get_embedding_file():
    model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP-PROYECTO/Word2Vec/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=400000)
    return model


def make_embedding_matrix(  words_idx_to_token, emb_dim):
    embeddings_gensim = get_embedding_file()
    final_embeddings = np.zeros((len(words_idx_to_token), emb_dim))
    for idx in sorted(words_idx_to_token):
        if words_idx_to_token[idx] in embeddings_gensim.key_to_index:
            final_embeddings[idx, :] = embeddings_gensim[words_idx_to_token[idx]]
        else:
            embedding_i = torch.ones(1, emb_dim)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[idx, :] = embedding_i
            print(f"Word {words_idx_to_token[idx]} not in model")

    return final_embeddings

In [30]:
args = Namespace(

    # Data and Path hyper parameters
    news_csv="canciones.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model_storage/model.pth",
    save_dir="model_storage/document_classification",


    # Model hyper parameters
    use_cnn_nlp = True,
    use_emb=True,
    embedding_size=300, #Depende del modelo de word embeddings que usemos. Alternativa: 300
    hidden_dim=100,
    num_channels=100, #AKA number of filters


    # Training hyper parameter
    seed=133,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,


    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
)

In [31]:
# Verificar si CUDA está disponible
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Establecer semilla para reproducibilidad
set_seed_everywhere(args.seed, args.cuda)

# Cargar dataset y manejar valores faltantes en 'lyrics'
songs_df = pd.read_csv(args.news_csv)
songs_df['lyrics'] = songs_df['lyrics'].fillna("").astype(str)  # Cambiar 'title' por 'lyrics'

# Crear dataset y vectorizador
dataset = SongDataset(songs_df, SongVectorizer.from_dataframe(songs_df))  # Cambiar 'title' por 'lyrics'
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Configuración de embeddings
if args.use_emb:
    review_vocab = vectorizer.lyrics_vocab  # Cambiar 'title_vocab' por 'lyrics_vocab'
    print(review_vocab)
    embeddings = make_embedding_matrix(
        review_vocab._idx_to_token, args.embedding_size
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None


Se han truncado las últimas 5000 líneas del flujo de salida.


and not in model
Word do


but not in model
Word lovers
because not in model
Word now
touch not in model
Word ga
radio not in model
Word close
in not in model
Word note
remind not in model
Word old
to not in model
Word setting
the not in model
Word shadows
and not in model
Word yourself
then not in model
Word now
yet not in model
Word echo
of not in model
Word journey
to not in model
Word while
come not in model
Word daddy
we not in model
Word death

the not in model
Word beyond
the not in model
Word man

he not in model
Word krishna not in model
Word you
stand not in model
Word divine
touched not in model
Word ambrosial not in model
Word friends
about not in model
Word weaknesses
i not in model
Word when
and not in model
Word baby
love not in model
Word impossible
to not in model
Word complainin
you not in model
Word spine
how not in model
Word rape
date not in model
Word hill
the not in model
Word nothing

you not in mode

In [32]:
if not args.use_cnn_nlp:
    classifier = NewsClassifier(
        embedding_size=args.embedding_size,
        num_embeddings=len(vectorizer.lyrics_vocab),  # Cambiar title_vocab por lyrics_vocab
        num_channels=args.num_channels,  # Esto es específico del modelo NewsClassifier
        hidden_dim=args.hidden_dim,
        num_classes=len(vectorizer.tag_vocab),  # Cambiar category_vocab por tag_vocab
        dropout_p=args.dropout_p,
        pretrained_embeddings=embeddings,
        padding_idx=0
    )
else:
    classifier = CNN_NLP(
        pretrained_embedding=embeddings,
        freeze_embedding=False,
        vocab_size=len(vectorizer.lyrics_vocab),  # Cambiar title_vocab por lyrics_vocab
        embed_dim=args.embedding_size,
        num_classes=len(vectorizer.tag_vocab)  # tag_vocab en lugar de category_vocab
    )


In [35]:
from sklearn.metrics import classification_report, accuracy_score
from torchinfo import summary

# Configuración del dispositivo
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {args.device}")

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min', factor=0.5,
                                                  patience=1)

train_state = make_train_state(args)

summary_shown = False

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Entrenamiento
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        all_y_true_train = []
        all_y_pred_train = []
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # Mostrar resumen del modelo solo una vez
            if not summary_shown:
                dummy_input = batch_dict['x_data'].to(args.device).long()
                print(summary(classifier, input_data=dummy_input))
                summary_shown = True

            # Paso 1: Resetear gradientes
            optimizer.zero_grad()

            # Paso 2: Salidas del modelo
            x_data = batch_dict['x_data'].to(args.device).long()
            y_pred = classifier(x_data)

            # Paso 3: Calcular pérdida
            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # Paso 4: Backpropagation
            loss.backward()

            # Paso 5: Actualizar pesos
            optimizer.step()

            # Cálculo de métricas
            y_pred_labels = y_pred.argmax(dim=1)
            all_y_true_train.extend(batch_dict['y_target'].cpu().tolist())
            all_y_pred_train.extend(y_pred_labels.cpu().tolist())
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        print(f"Epoch {epoch_index} - Train Loss: {running_loss:.4f}, Accuracy: {running_acc:.4f}")

        # Reporte de entrenamiento
        print("\nResultados en el conjunto de entrenamiento:")
        print(classification_report(all_y_true_train, all_y_pred_train, target_names=vectorizer.tag_vocab._token_to_idx.keys()))

        # Validación
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        all_y_true_val = []
        all_y_pred_val = []
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            x_data = batch_dict['x_data'].to(args.device).long()
            y_pred = classifier(x_data)

            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            y_pred_labels = y_pred.argmax(dim=1)
            all_y_true_val.extend(batch_dict['y_target'].cpu().tolist())
            all_y_pred_val.extend(y_pred_labels.cpu().tolist())
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        print(f"Epoch {epoch_index} - Val Loss: {running_loss:.4f}, Accuracy: {running_acc:.4f}")

        # Reporte de validación
        print("\nResultados en el conjunto de validación:")
        print(classification_report(all_y_true_val, all_y_pred_val, target_names=vectorizer.tag_vocab._token_to_idx.keys()))

        scheduler.step(running_loss)

except KeyboardInterrupt:
    print("Training interrupted.")


Using device: cuda
Layer (type:depth-idx)                   Output Shape              Param #
CNN_NLP                                  [128, 4]                  --
├─Embedding: 1-1                         [128, 5450, 300]          15,798,600
├─ModuleList: 1-2                        --                        --
│    └─Conv1d: 2-1                       [128, 100, 5448]          90,100
│    └─Conv1d: 2-2                       [128, 100, 5447]          120,100
│    └─Conv1d: 2-3                       [128, 100, 5446]          150,100
├─Dropout: 1-3                           [128, 300]                --
├─Linear: 1-4                            [128, 4]                  1,204
Total params: 16,160,104
Trainable params: 16,160,104
Non-trainable params: 0
Total mult-adds (G): 253.22
Input size (MB): 5.58
Forward/backward pass size (MB): 3347.56
Params size (MB): 64.64
Estimated Total Size (MB): 3417.78
Epoch 0 - Train Loss: 0.0881, Accuracy: 97.9421

Resultados en el conjunto de entrenamiento:


# **LSTM**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SongLSTMClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings,
                 hidden_dim, num_classes, dropout_p, batch_size,
                 pretrained_embeddings=None, padding_idx=0):
        """
        Modelo LSTM para clasificar géneros musicales basándose en las letras de canciones.

        Args:
            embedding_size (int): Tamaño de los vectores de embedding.
            num_embeddings (int): Número de palabras en el vocabulario.
            hidden_dim (int): Tamaño de la capa oculta del LSTM.
            num_classes (int): Número de géneros posibles (clases de salida).
            dropout_p (float): Probabilidad de dropout para regularización.
            batch_size (int): Tamaño de batch.
            pretrained_embeddings (numpy.array): Matriz de embeddings preentrenados (opcional).
            padding_idx (int): Índice que representa posiciones nulas (padding).
        """
        super(SongLSTMClassifier, self).__init__()

        # Embedding layer: carga embeddings preentrenados o inicializa aleatoriamente
        if pretrained_embeddings is None:
            self.embedding = nn.Embedding(
                num_embeddings=num_embeddings,
                embedding_dim=embedding_size,
                padding_idx=padding_idx
            )
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.embedding = nn.Embedding(
                num_embeddings=num_embeddings,
                embedding_dim=embedding_size,
                padding_idx=padding_idx,
                _weight=pretrained_embeddings
            )

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        # LSTM layer: Modela las dependencias de las secuencias
        self.lstm = nn.LSTM(
            input_size=embedding_size,  # Dimensión de entrada del LSTM
            hidden_size=hidden_dim,    # Dimensión de salida del LSTM
            num_layers=1,              # Una capa LSTM
            batch_first=True           # Dimensiones del batch primero (batch, seq, feature)
        )

        # Dropout: Ayuda a evitar el overfitting
        self.dropout = nn.Dropout(p=dropout_p)

        # Fully connected layer: Reduce la salida del LSTM a num_classes
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """
        Realiza el paso forward del modelo.

        Args:
            x_in (torch.Tensor): Tensor de entrada (batch, seq_len).
            apply_softmax (bool): Flag para aplicar softmax en la salida.
        Returns:
            torch.Tensor: Predicciones del modelo (batch, num_classes).
        """
        # Embedding
        embedded = self.embedding(x_in)  # (batch, seq_len, embedding_size)

        # LSTM
        output, (hidden, cell) = self.lstm(embedded)  # output: (batch, seq_len, hidden_dim)

        # Seleccionar la última salida del LSTM
        last_hidden_state = output[:, -1, :]  # (batch, hidden_dim)

        # Dropout
        dropped = self.dropout(last_hidden_state)

        # Fully connected layer
        prediction_vector = self.fc(dropped)  # (batch, num_classes)

        # Softmax (si se requiere)
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector


In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings

    Args:
        glove_filepath (str): path to the glove embeddings file
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, "r") as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0]
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.

    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]

    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            print(f"Word {word} is not in Glove")
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    news_csv="canciones.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model_lstm.pth",
    save_dir="model_storage/document_classification_lstm",
    # Model hyper parameters
    glove_filepath='/content/drive/MyDrive/NLP-PROYECTO/embeddings_lstm/glove.6B.100d.txt',
    #glove_filepath='/content/drive/MyDrive/Cursos impartidos/NLP/Embeddings/glove.840B.300d.txt',
    use_glove=True,
    embedding_size=100,
    hidden_dim=100,
    # Training hyper parameter
    seed=1337,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
)


# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)


Using CUDA: True


In [ ]:
# create dataset and vectorizer
dataset = SongDataset.load_dataset_and_make_vectorizer(args.news_csv)
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

if args.use_glove:
    words = vectorizer.lyrics_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(
        glove_filepath=args.glove_filepath,
        words=words
    )
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None



classifier = SongLSTMClassifier(embedding_size=args.embedding_size, num_embeddings=len(vectorizer.lyrics_vocab),hidden_dim=args.hidden_dim,num_classes=len(vectorizer.tag_vocab),batch_size=args.batch_size, dropout_p=args.dropout_p,pretrained_embeddings=embeddings, padding_idx=0)

<ipython-input-11-31475dafa5b2>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs_df['lyrics'] = songs_df['lyrics'].fillna("").astype(str)


Streaming output truncated to the last 5000 lines.
mystic is not in Glove
Word my
torture is not in Glove
Word had
how is not in Glove
Word radio
it is not in Glove
Word skrt
skrt is not in Glove
Word some
yeah is not in Glove
Word js
okay is not in Glove
Word okay
dropped is not in Glove
Word porsche
lost is not in Glove
Word no
stupid is not in Glove
Word would
if is not in Glove
Word understand

it is not in Glove
Word up
makes is not in Glove
Word sweet
it is not in Glove
Word time
did is not in Glove
Word loud
choppa is not in Glove
Word loudest
choppa is not in Glove
Word yeah
type is not in Glove
Word more
tonight is not in Glove
Word spain
i is not in Glove
Word move
make is not in Glove
Word dbtrap is not in Glove
Word ey
throw is not in Glove
Word breasts
i is not in Glove
Word roads
come is not in Glove
Word ahhhh
i is not in Glove
Word hussl is not in Glove
Word rhino
i is not in Glove
Word flow
yeah is not in Glove
Word crusin is not in Glove
Word me
lay is not in Glove
Wo

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary

# Configuración correcta del resumen
dummy_input = torch.randint(0, vocab_size, (1, 100)).to(args.device).long()  # Simula datos válidos
summary(classifier, input_data=dummy_input)


classifier = classifier.to(args.device)
summary(classifier, input_size=(100,), device=str(args.device))
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm_notebook(desc='training routine',
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm_notebook(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size),
                          position=1,
                          leave=True)
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size),
                        position=1,
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        all_y_true_train = []
        all_y_pred_train = []
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # Training steps
            optimizer.zero_grad()
            y_pred = classifier(batch_dict['x_data'])
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss.backward()
            optimizer.step()

            # Loss and accuracy
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # Collect true and predicted values
            y_pred_labels = y_pred.argmax(dim=1)
            all_y_true_train.extend(batch_dict['y_target'].cpu().tolist())
            all_y_pred_train.extend(y_pred_labels.cpu().tolist())

            # Update train bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc,
                                  epoch=epoch_index)
            train_bar.update()

        # Compute additional metrics for training
        train_precision = precision_score(all_y_true_train, all_y_pred_train, average='weighted')
        train_recall = recall_score(all_y_true_train, all_y_pred_train, average='weighted')
        train_f1 = f1_score(all_y_true_train, all_y_pred_train, average='weighted')
        print(f"Epoch {epoch_index} - Train Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-Score: {train_f1:.4f}")

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        all_y_true_val = []
        all_y_pred_val = []
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # Validation steps
            y_pred = classifier(batch_dict['x_data'])
            loss = loss_func(y_pred, batch_dict['y_target'])

            # Loss and accuracy
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # Collect true and predicted values
            y_pred_labels = y_pred.argmax(dim=1)
            all_y_true_val.extend(batch_dict['y_target'].cpu().tolist())
            all_y_pred_val.extend(y_pred_labels.cpu().tolist())

            # Update val bar
            val_bar.set_postfix(loss=running_loss, acc=running_acc,
                                epoch=epoch_index)
            val_bar.update()

        # Compute additional metrics for validation
        val_precision = precision_score(all_y_true_val, all_y_pred_val, average='weighted')
        val_recall = recall_score(all_y_true_val, all_y_pred_val, average='weighted')
        val_f1 = f1_score(all_y_true_val, all_y_pred_val, average='weighted')
        print(f"Epoch {epoch_index} - Validation Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])
        print(f"Early stopping state is {train_state['early_stopping_step']}")
        if train_state['stop_early']:
            break

        # Reset bars for the next epoch
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

except KeyboardInterrupt:
    print("Exiting loop")


NameError: name 'vocab_size' is not defined

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:

print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 1.3865471158708846;
Test Accuracy: 23.861607142857142
